# Import

In [1]:
// %install-swiftpm-flags -c release
// %install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.5")' SwiftCoreMLTools
// %install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

In [2]:
import Foundation
import TensorFlow
// import SwiftCoreMLTools
// import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let embeddingSizes = allCategoriesValues.map{ $0.count }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
let oneHotCategoricalFeatures:[[[Int32]]] = categoricalFeatures.map{ catArray in
    var oneHotArray = [[Int32]]()
    
    for i in 0..<catArray.count {
        var oneHot = Array(repeating: Int32(0), count: allCategoriesValues[i].count)
        if let pos = allCategoriesValues[i].firstIndex(where: { $0 == catArray[i] }){
            oneHot[pos] = 1
        }
        oneHotArray.append(oneHot)
    }
    
    return oneHotArray
}

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Split Train and Test

In [6]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(oneHotCategoricalFeatures[0..<numTrainRecords])).map{ Array($0.joined()) }
let xCategoricalAllTest = matrixTranspose(Array(oneHotCategoricalFeatures[numTrainRecords...])).map{ Array($0.joined()) }
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, embeddingSizes[offset]]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, embeddingSizes[offset]]))
}
let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [7]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 2.0137098,  14.197531,   9.523555, 0.53213036,  6.3311296,   64.47929,  4.1678762,  353.68396,
    18.03163,  379.84735,  11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.11449408,  0.7311985,  29.000755,  2.1797554,  132.14561,
    2.217345,  40.494495,   6.852825]]


In [8]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [9]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 2] [405, 9] [405, 1]
Testing shapes  [101, 11] [101, 2] [101, 9] [101, 1]


# Model

In [23]:
struct MultiInputs<N: Differentiable, C: Differentiable>: Differentiable {
  var numerical: [N]
  var categorical: [C]

  @differentiable
  init(numerical: [N], categorical: [C]) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Layer {
    var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 4)
    var layer1 = Dense<Float>(inputSize: 11, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<Tensor<Float>, Tensor<Float>>) -> Tensor<Float> {
        return input.numerical[0].sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [9]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [10]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [11]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 589.81085, MAE: 21.91811
Epoch 2: MSE: 532.30084, MAE: 20.598415
Epoch 3: MSE: 471.52426, MAE: 19.115623
Epoch 4: MSE: 406.53738, MAE: 17.479641
Epoch 5: MSE: 338.0967, MAE: 15.640751
Epoch 6: MSE: 273.65936, MAE: 13.59707
Epoch 7: MSE: 214.2916, MAE: 11.672209
Epoch 8: MSE: 167.4195, MAE: 9.808268
Epoch 9: MSE: 131.0001, MAE: 8.390486
Epoch 10: MSE: 103.84559, MAE: 7.331494
Epoch 11: MSE: 84.44268, MAE: 6.512589
Epoch 12: MSE: 69.91607, MAE: 5.7860866
Epoch 13: MSE: 57.865364, MAE: 5.234492
Epoch 14: MSE: 48.205647, MAE: 4.7263517
Epoch 15: MSE: 41.01215, MAE: 4.35292
Epoch 16: MSE: 35.789993, MAE: 3.9689064
Epoch 17: MSE: 32.554733, MAE: 3.7550542
Epoch 18: MSE: 29.312397, MAE: 3.5553021
Epoch 19: MSE: 27.672157, MAE: 3.4326134
Epoch 20: MSE: 26.412117, MAE: 3.3594458
Epoch 21: MSE: 24.803968, MAE: 3.229053
Epoch 22: MSE: 23.748573, MAE: 3.1771557
Epoch 23: MSE: 22.921827, MAE: 3.097651
Epoch 24: MSE: 21.684204, MAE: 2.9909055
Epoch 25: MSE: 21.041624, MAE: 2.910099
Epo

Epoch 205: MSE: 3.8929899, MAE: 1.3233553
Epoch 206: MSE: 3.9347196, MAE: 1.3088499
Epoch 207: MSE: 4.1642003, MAE: 1.3574431
Epoch 208: MSE: 3.9191463, MAE: 1.3176068
Epoch 209: MSE: 4.0800037, MAE: 1.3158011
Epoch 210: MSE: 3.86485, MAE: 1.2927513
Epoch 211: MSE: 3.7487135, MAE: 1.2933971
Epoch 212: MSE: 4.255821, MAE: 1.3012805
Epoch 213: MSE: 3.6888735, MAE: 1.2751573
Epoch 214: MSE: 4.1169477, MAE: 1.3376248
Epoch 215: MSE: 3.8661308, MAE: 1.301401
Epoch 216: MSE: 3.880018, MAE: 1.3036292
Epoch 217: MSE: 3.762938, MAE: 1.2912028
Epoch 218: MSE: 3.7680612, MAE: 1.273327
Epoch 219: MSE: 3.7489762, MAE: 1.2884958
Epoch 220: MSE: 4.0362253, MAE: 1.3122103
Epoch 221: MSE: 3.8192155, MAE: 1.3059095
Epoch 222: MSE: 3.645543, MAE: 1.253865
Epoch 223: MSE: 3.8001466, MAE: 1.2974814
Epoch 224: MSE: 3.7164667, MAE: 1.2707146
Epoch 225: MSE: 3.8923404, MAE: 1.2865106
Epoch 226: MSE: 3.6474395, MAE: 1.2795918
Epoch 227: MSE: 3.8348398, MAE: 1.2878511
Epoch 228: MSE: 3.6273088, MAE: 1.2543614
E

Epoch 402: MSE: 2.5191193, MAE: 1.0958669
Epoch 403: MSE: 2.3870103, MAE: 1.064272
Epoch 404: MSE: 2.6042123, MAE: 1.1122155
Epoch 405: MSE: 2.2502027, MAE: 1.0423396
Epoch 406: MSE: 2.2356317, MAE: 1.0436115
Epoch 407: MSE: 2.546919, MAE: 1.1543307
Epoch 408: MSE: 2.2689524, MAE: 1.0468328
Epoch 409: MSE: 2.4096115, MAE: 1.1191856
Epoch 410: MSE: 2.4195154, MAE: 1.0729276
Epoch 411: MSE: 2.2556474, MAE: 1.0638112
Epoch 412: MSE: 2.6347125, MAE: 1.0511119
Epoch 413: MSE: 2.2540889, MAE: 1.0044751
Epoch 414: MSE: 2.297465, MAE: 1.0993241
Epoch 415: MSE: 2.5899913, MAE: 1.1363679
Epoch 416: MSE: 2.349672, MAE: 1.0532255
Epoch 417: MSE: 2.2123332, MAE: 1.0402236
Epoch 418: MSE: 2.468155, MAE: 1.0614207
Epoch 419: MSE: 2.2284586, MAE: 0.99371296
Epoch 420: MSE: 2.4191208, MAE: 1.1322768
Epoch 421: MSE: 2.3195207, MAE: 1.0351545
Epoch 422: MSE: 2.3122172, MAE: 1.1503565
Epoch 423: MSE: 2.1746047, MAE: 1.0312278
Epoch 424: MSE: 2.2582936, MAE: 1.0885746
Epoch 425: MSE: 2.1674268, MAE: 1.0763

# Test

In [12]:
Context.local.learningPhase = .inference

let prediction = model(XTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.8232478, MAE: 0.070073694


In [13]:
print(XTrain[0], YTrain[0], model(XTrain[0].reshaped(to: TensorShape([1, 13]))))

print(XTest[0], YTest[0], model(XTest[0].reshaped(to: TensorShape([1, 13]))))
print(XTest[1], YTest[1], model(XTest[1].reshaped(to: TensorShape([1, 13]))))
print(XTest[17], YTest[17], model(XTest[17].reshaped(to: TensorShape([1, 13]))))

[ -0.30846816,   0.15054052,   -1.1039964,  -0.30756173,   0.05126577,   0.33352128,
  0.024851447, -0.035726987,  -0.89119065,  -0.43651816,   -1.2319369,   0.42111015,
  -0.93603975] [24.0] [[22.523558]]
[  10.127699, -0.56208307,   1.3125796, -0.30756173,   1.4050479,  -0.8863933,   1.2248203,
  -1.2581577,   2.6233904,   2.3634233,   0.9779132,  0.12650238,   1.6906141] [5.0] [[2.0903258]]
[  2.8739426, -0.56208307,   1.3125796, -0.30756173,    1.108089,  -2.9993627,   1.2248203,
  -1.3716108,   2.6233904,   2.3634233,   0.9779132, -0.23774466,   1.7431473] [11.9] [[15.41979]]
[   1.541963, -0.56208307,   1.3125796, -0.30756173,   0.7150559, -0.93426037,   0.7972452,
  -1.0169381,   2.6233904,   2.3634233,   0.9779132,   -2.180478,  0.39479825] [20.8] [[32.461395]]


# Export

In [14]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

[13, 64] [64, 32] [32, 1]
[64] [32] [1]


In [15]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13])
    Output(name: "output", shape: [1])
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weight: model.layer1.weight.transposed().flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weight: model.layer2.weight.transposed().flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weight: model.layer3.weight.transposed().flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1)
    }
}

In [16]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))